# ДТП с участием такси

**УСЛОВИЕ ЗАДАЧИ** <br>
В городе N произошло ДТП с наездом такси на пешехода. Известно, что в городе только два цвета машин - белые и желтые. Причем желтых 85%, остальные белые. Есть свидетель, который говорит, что в ДТП участвовала желтая машина. Однако, так как наезд совершен ночью, свидетель может заблуждаться в своих показаниях, и в темное время суток в 20% случаев он путает желтый цвет с белым. Какова вероятность того, что ДТП было с участием желтой машины?

Решим задачу несколькими способами:

**1. Статистически**<br>

**2. Используя классическое определение вероятности**

**3. Используя формулу Байеса**





## Статистический метод

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

Зададим указанные в условии частоты

In [2]:
# частота ошибки в цвете 
false_color = 0.20

# частота, с которой можно встретить желтое такси
yellow_car = 0.15

Моделируем желтые и белые такси, из расчета указанных частот и 10 млн. количества

In [3]:
# список желтых (1) и белых (0) такси в количестве 10 млн. с частотой 0.15 желтые и 0.85 белые
yellow_white = [np.random.binomial(1, yellow_car) for _ in tqdm(range(10**7))]
len(yellow_white)

  0%|          | 0/10000000 [00:00<?, ?it/s]

10000000

Теперь, для моделирования частоты, с которой свидетель верно определяет желтые цвет такси создадим функцию test_color

In [4]:
# моделируем результаты определения желтого цвета (0 - ошибка, 1 - верно)
def test_color(col):
    if np.random.binomial(1, 1 - false_color):
        return col
    return abs(col-1)

# сразу проверим
print('В случае с желтым (верно определить):', round(np.mean([test_color(1) for _ in range(10000)]), 2))
print('В случае с белым (совершить ошибку):', round(np.mean([test_color(0) for _ in range(10000)]), 2))

В случае с желтым (верно определить): 0.81
В случае с белым (совершить ошибку): 0.2


Готово. Теперь соберем полученные данные вместе и смоделируем город N

In [ ]:
# применяем функцию частоты, с которой свидетель определяет желтый цвет машины
test_taxi = []

for x in tqdm(yellow_white):
    taxi_color = test_color(x)
    test_taxi.append((x, taxi_color))

In [ ]:
# создаем датафрейм city_N
city_N = pd.DataFrame(test_taxi, columns=['color', 'test_color'])

In [43]:
# группируем по результату определения цвета такси
print('Вероятность того, что в ДТП участвовала желтая машина примерно', 
      round(city_N.groupby('test_color')['color'].mean().iloc[1], 4))

Вероятность того, что в ДТП участвовала желтая машина примерно 0.4141


## Классическое определение вероятности

Пусть событие А - свидетель верно определил желтый цвет машины, pA - вероятность события А.

Рассмотрим город N, в котором 100 мишин такси (без ограничения общности можем взять любое число). Рассчитаем сколько белых и желтых машин из 100 будут правильно/неправильно определены свидетелем.

In [32]:
count_taxi = 100

white_t = 0.85
yellow_t = 1 - white_t

test_ye_color_true = 0.8
test_ye_color_false = 1 - test_ye_color_true

taxi_white = round(count_taxi * white_t)
taxi_yellow = round(count_taxi * yellow_t)

print('Всего такси в городе N:', count_taxi)
print('Белых такси в городе N:', taxi_white)
print('Желтых такси в городе N:', taxi_yellow)

Всего такси в городе N: 100
Белых такси в городе N: 85
Желтых такси в городе N: 15


In [39]:
test_true_w = round(taxi_white * test_ye_color_true)
test_false_w = round(taxi_white * test_ye_color_false)


test_true_y = round(taxi_yellow * test_ye_color_true)
test_false_y = round(taxi_yellow * test_ye_color_false)

print("Из {} белых машин свидетель правильно определил цвет {}, а {} машин посчитал желтыми"
      .format(taxi_white,test_true_w,test_false_w))

print("Из {} желтых машин свидетель правильно определил цвет {}, а {} машин посчитал белыми"
      .format(taxi_yellow,test_true_y,test_false_y))

Из 85 белых машин свидетель правильно определил цвет 68, а 17 машин посчитал желтыми
Из 15 желтых машин свидетель правильно определил цвет 12, а 3 машин посчитал белыми


Вероятность того, что в ДТП участвовала желтая машина, можно найти как отношение количества желтых машин, которые свидетель определил верно, к числу всех машин, что он мог посчитать желтыми.

In [46]:
pA = round(test_true_y/(test_true_y + test_false_w), 4)
print("Вероятность того, что свидетель точно определил желтый цвет машины равна {} или {}/{}"
      .format(pA, test_true_y, test_true_y + test_false_w))

Вероятность того, что свидетель точно определил желтый цвет машины равна 0.4138 или 12/29


## Формула Байеса

Напомним формулу Байеса 

$$ P(A|B) = \frac{P(B|A)P(A)}{P(B)} $$

*P(A|B)* - вероятность того, что произошло событие *А*, при условии наступления события *В* <br>
*P(B|A)* - вероятность того, что произошло событие *В*, при условии наступления события *А* <br>
*P(A)* - вероятность события *А*, *P(B)* - вероятность события *B*.

**Пусть в нашей задаче:**

Событие *А* - такси желтое, *В* - свидетель определил цвет такси как желтый. <br>
*P(A|B)* - вероятность того, что такси действительно желтое если так сказал свидетель. <br>
*P(A)* - вероятность того, что такси желтое. <br>
*P(B|A)* - вероятность того, что свидетель определил такси как желтое при условии, что оно действительно желтое.<br>
*P(B)* - вероятность того, что свидетель определил такси как желтое.

**Имеем:**

*P(A) = 0.15*, *P(B|A) = 0.8*, *P(B)* надо найти. Используем для этого формулу полной вероятности.<br>

$$ P(B) = P(B|A)P(A)+P(B|A')P(A')$$

Где *A'* - событие, обратное *А*.

In [57]:
# По условию задачи
# вероятность, с которой встречаются желтые такси
p_A = 0.15
# вероятность, с которой свидетель правильно определяет желтый цвет такси
p_BA = 0.8
# вероятность, с которой свидетель неправильно определяет желтый цвет такси
p_BnotA = 0.2
# вероятность, с котрой встречается белое такси
p_notA = 0.85

# вероятность, с которой свидетель определяет такси как желтое
p_B = p_BA * p_A + p_BnotA * p_notA

print('Вероятность, с которой свидетель определяет такси как желтое:', round(p_B,4))

Вероятность, с которой свидетель определяет такси как желтое: 0.29


In [58]:
# вероятность того, что такси действительно желтое, как и сказал свидетель
p_AB = (p_BA * p_A) / p_B

print('Вероятность, с которой такси оказалось желтым, как и сказал свидетель:', round(p_AB, 4))

Вероятность, с которой такси оказалось желтым, как и сказал свидетель: 0.4138
